# 数据部分

In [1]:
movie_path = "./data/ml-1m/movies.dat"
user_path = "./data/ml-1m/users.dat"
rating_path = "./data/ml-1m/ratings.dat"

In [2]:
from sys import path
path.append("./data/ml-1m/")
import data_helper
from imp import reload
reload(data_helper)



<module 'data_helper' from './data/ml-1m/data_helper.py'>

##  电影数据

格式:
>MovieID::Title::Genres

- Titles are identical to titles provided by the IMDB (including
year of release)
- Genres are pipe-separated and are selected from the following genres:

        * Action
        * Adventure
        * Animation
        * Children's
        * Comedy
        * Crime
        * Documentary
        * Drama
        * Fantasy
        * Film-Noir
        * Horror
        * Musical
        * Mystery
        * Romance
        * Sci-Fi
        * Thriller
        * War


In [3]:
movies = data_helper.read_movies(movie_path)

## 用户数据

格式:
>UserID::Gender::Age::Occupation::Zip-code

- Gender is denoted by a "M" for male and "F" for female
- Age is chosen from the following ranges:

        *  1:  "Under 18"
        * 18:  "18-24"
        * 25:  "25-34"
        * 35:  "35-44"
        * 45:  "45-49"
        * 50:  "50-55"
        * 56:  "56+"

- Occupation is chosen from the following choices:

        *  0:  "other" or not specified
        *  1:  "academic/educator"
        *  2:  "artist"
        *  3:  "clerical/admin"
        *  4:  "college/grad student"
        *  5:  "customer service"
        *  6:  "doctor/health care"
        *  7:  "executive/managerial"
        *  8:  "farmer"
        *  9:  "homemaker"
        * 10:  "K-12 student"
        * 11:  "lawyer"
        * 12:  "programmer"
        * 13:  "retired"
        * 14:  "sales/marketing"
        * 15:  "scientist"
        * 16:  "self-employed"
        * 17:  "technician/engineer"
        * 18:  "tradesman/craftsman"
        * 19:  "unemployed"
        * 20:  "writer"


In [4]:
users = data_helper.read_users(user_path)

## 点击数据

格式：
>serID::MovieID::Rating::Timestamp

    - UserIDs range between 1 and 6040
    - MovieIDs range between 1 and 3952
    - Ratings are made on a 5-star scale (whole-star ratings only)
    - Timestamp is represented in seconds since the epoch as returned by time(2)
    - Each user has at least 20 ratings


In [3]:
train_set,test_set = data_helper.read_ratings(rating_path)

### 下采样

In [4]:
import random

trainset = dict()
testset = dict()

t = random.sample(train_set.keys(),100)
for k in t:trainset[k] = train_set[k]
t = random.sample(test_set.keys(),250)
for k in t:testset[k] = test_set[k]
    
del k,t,train_set,test_set

## 分离训练集和测试集

# 协同过滤

In [41]:
import random

def splitData(data,M,k,seed):
    """
        将数据分成训练集和测试集
        Args:
            data :   数据
            M :   一共分的份数
            k :   第几份
            seed :    random的seed
        Return:
            训练集，测试集
    """
    train = dict()
    test = dict()
    
    random.seed(seed)
    for user,item in data.items():
        if random.randint(0,M) == k:
            test[user] = item
        else:
            train[user] = item
            
    return train,test
        

## 测评指标

对用户 u 推荐 N 个物品(记为 R ( u ) ),令用户 u 在测试集上喜欢的物品集合为 T ( u ) ,然后可以通
过准确率 / 召回率评测推荐算法的精度:
<br>
<br>
<br>
        $$Recall=\frac{\sum_u{\mid R(u)\bigcap T(u)}\mid}{\sum_u{\mid T(u) \mid} }$$
        $$Precision=\frac{\sum_u{\mid R(u)\bigcap T(u) \mid} }{\sum_u{\mid R(u) \mid}}$$

### Recall

In [39]:
def recall(trainSet,testSet,W,N):
    hit = 0.
    r = 0.
    for user,items in testSet.items():
        rank = recommend(user = user,train = trainSet,W = W,N = N)
        hit += len(set(items.keys()) & set(rank.keys()))
        r += len(items.keys())
        
    return hit / r

### Precision

In [45]:
def precision(trainSet,testSet,W,N):
    hit = 0.
    r = 0.
    for user,items in testSet.items():
        rank = recommend(user = user,train = trainSet,W = W,N = N)
        hit += len(set(items.keys()) & set(rank.keys()))
        r += len(rank.keys())
        
    return hit / r

### Coverage

>覆盖率反映了推荐算法发掘长尾的能力,覆盖率越高,说明推荐算法越能够将长尾中的物品推荐给用户。这里,我们采用最简单的覆盖率定义:
$$Coverage = \frac{\mid \bigcup_{u\in U} R(u) \mid}{\mid I \mid}$$
该覆盖率表示最终的推荐列表中包含多大比例的物品。如果所有的物品都被推荐给至少一个
用户,那么覆盖率就是 100% 。

In [9]:
def Coverage(train,test,N):
    """
    """
    recommend_items = set()
    all_items = set()
    for user in train.keys():
        for item in train[user].keys():
            all_items.add(item)
        rank = GetRecommendation(user,N)
        for item,pui in rank:
            recommend_item.add(item)
    return len(recommend_items) / (len(all_items) * 1.0)

### 新疑度

In [12]:
import math

def Popularity(train,test,N):
    item_popularity = dict()
    for user,items in train.items():
        for item in items.keys():
            if item not in item_popularity:
                item_popularity[item] = 0
            item_popularity[item] += 1
    ret = 0
    n = 0
    for user in train.keys():
        rank = GetRecommendation(user,N)
        for item,pui in rank:
            ret += math.log(1 + item_popularity[item])
            n += 1
    ret /= n * 1.0
    return ret

## 基于用户的的协同过滤

从上面的描述中可以看到,基于用户的协同过滤算法主要包括两个步骤。
1. 找到和目标用户兴趣相似的用户集合。
2. 找到这个集合中的用户喜欢的,且目标用户没有听说过的物品推荐给目标用户。

步骤 (1) 的关键就是计算两个用户的兴趣相似度。这里,协同过滤算法主要利用行为的相似度
计算兴趣的相似度。给定用户 u 和用户 v ,令 N(u) 表示用户 u 曾经有过正反馈的物品集合,令 N(v)
为用户 v 曾经有过正反馈的物品集合。那么,我们可以通过如下的 Jaccard 公式简单地计算 u 和 v 的
兴趣相似度:
$$w_{uv}=\frac{\mid N(u)\bigcap N(v) \mid}{\mid N(u) \bigcup N(v) \mid}$$
或者通过余弦相似度计算:
$$w_{uv}=\frac{\mid N(u)\bigcap N(v) \mid}{\sqrt{\mid N(u) \mid \mid  N(v) \mid}}$$

实际上，以上的方式也可以看作一个稀疏矩阵（列表示Item，行表示User）

<br>

|User\Item | a | b | c | d | e |
| - | :-: | -: | 
| A | 0 | 1 | 0 | 0 | 1 |
| B | 1 | 0 | 1 | 0 | 1 |
| C | 1 | 1 | 1 | 0 | 1 |
| D | 0 | 0 | 0 | 0 | 1 |



### 计算相似性矩阵

In [8]:
import math

def UserSimilarity(train):
    """计算相似性的矩阵"""
    W = dict()
    for u,u_items in train.items():
        for v,v_items in train.items():
            W.setdefault(u,{})
            union = set(u_items.keys()) & set(v_items.keys())
            if u == v : continue
            W[u][v] = len(union) / math.sqrt(len(u_items.keys()) * len(v_items.keys()))
            
    return W

### 相似性矩阵的改进

In [5]:
from collections import defaultdict
import math

def UserSimilarity(train):
    """计算相似性矩阵"""
    W = dict()
    
    item2user = defaultdict(set)
    for user,items in train.items():
        for item in items.keys():
            item2user[item].add(user)
            
    N = defaultdict(int)
    C = dict()
    
    for item,users in item2user.items():
        for u in users:
            N[u] += 1
            C.setdefault(u,defaultdict(int)) 
            for v in users:
                if u == v:
                    continue
                C[u][v] += 1    
    W = dict()
    for u,uc in C.items():
        for v,c in uc.items():
            W.setdefault(u,defaultdict(float))
            W[u][v] = C[u][v] / math.sqrt(N[u] * N[v])
            
    return W

### 推荐函数主体

In [43]:
from collections import defaultdict
from operator import itemgetter

def recommend(user,train,W,N = 10,K = 80):
    """给user推荐商品,N表示推荐前多少个，K表示找最近的K的用户，W表示相似性矩阵"""
    rank = defaultdict(int)
    #得到user已经购买的商品
    user_items = train.get(user,{})
    for v,similarity in sorted(W.get(user,{}).items(),key = itemgetter(1))[:K]:
        for item,rate in train[v].items():
            if item in user_items: continue
            rank[item] += similarity * rate
            
    #对结果排序
    rank = sorted(rank.items(),key = itemgetter(1),reverse = True)[:N]
    return dict(rank)
            

### 测试

In [47]:
if __name__ == "__main__":
    #获取相似性的矩阵
    W = UserSimilarity(train = trainset)
    
    recall_s = recall(trainSet = trainset,testSet = testset,W = W,N = 20)
    precision_s = precision(trainSet = trainset,testSet = testset,W = W,N = 20)
    
    print("Recall : {}".format(recall_s))
    print("Precision : {}".format(precision_s))

Recall : 0.006844064826982041
Precision : 0.06188118811881188


## 基于Item的协同过滤

### Item的相似性矩阵

In [5]:
from collections import defaultdict
import math

def ItemSimilarity(train):
    """"""
    movie2user = dict()
    for user,movies in train.items():
        for movie,rating in movies.items():
            movie2user.setdefault(movie,set())
            movie2user[movie].add(user)
   
    W = dict()
    for i,i_users in movie2user.items():
        W.setdefault(i,{})
        for j,j_users in movie2user.items():
            if i == j:
                continue
            #统计交集的数量
            union_count = len(i_users & j_users)
            W[i][j] = union_count / math.sqrt(len(i_users) *len(j_users))
        
    return W

### 测试

In [6]:
if __name__ == "__main__":
    W = ItemSimilarity(trainset)